In [109]:
import sklearn
import time
import pandas as pd   
from matplotlib import pyplot
from numpy import mean
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB, CategoricalNB
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier



In [207]:
def robust_scale(df):
    numerical_columns = df.select_dtypes(include=['number']).columns
    scaler = RobustScaler()
    df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

    return df

def knn_impute_numerical_columns(df, n_neighbors=10):
    df_imputed = df.copy()
    # df_imputed = pd.get_dummies(drop_first=True, data=df_imputed)
    
    categorical_columns = df_imputed.select_dtypes(exclude=['number']).columns
    numerical_cols = df_imputed.select_dtypes(include=['number']).columns
    
    imputer = KNNImputer(n_neighbors=n_neighbors)
    # imputer = SimpleImputer(strategy='mean') 
    df_imputed[numerical_cols] = imputer.fit_transform(df_imputed[numerical_cols])
    
    # Label encode categorical columns
    # label_encoders = {}
    # for col in categorical_columns:
    #     label_encoders[col] = LabelEncoder()
    #     df_imputed[col] = label_encoders[col].fit_transform(df_imputed[col])

    # imputer = SimpleImputer(strategy='constant', fill_value='Unknown')
    imputer = SimpleImputer(strategy='most_frequent')
    df_imputed[categorical_columns] = imputer.fit_transform(df_imputed[categorical_columns])
    
    # cat_imputer = SimpleImputer(strategy='most_frequent')
    # df_imputed[categorical_columns] = cat_imputer.fit_transform(df_imputed[categorical_columns])

    return df_imputed


In [208]:
df = pd.read_csv('train.csv')



In [209]:
robust_scale(df)

,RecordID,hospital_id,icu_id,ethnicity,gender,icu_admit_source,icu_stay_type,icu_type,apache_3j_bodysystem,apache_2_bodysystem,...,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,d1_glucose_max,d1_potassium_max,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,immunosuppression,solid_tumor_with_metastasis,hospital_death
0,-1.00000,-0.666667,1.241270,Caucasian,M,Floor,transfer,Med-Surg ICU,Metabolic,Metabolic,...,-0.805556,-1.000000,-0.805556,NaN,NaN,-0.333333,-0.333333,0.0,0.0,0.0
1,-0.99996,-0.789474,0.012698,African American,M,Accident & Emergency,admit,Med-Surg ICU,Cardiovascular,Cardiovascular,...,-0.555556,-0.945946,-0.555556,2.164706,-0.500,NaN,NaN,0.0,0.0,0.0
2,-0.99992,-0.429825,-0.073016,Caucasian,M,Floor,admit,MICU,Respiratory,Respiratory,...,1.305556,1.189189,1.305556,0.129412,0.125,2.750000,2.166667,0.0,0.0,0.0
3,-0.99988,-0.815789,0.860317,Caucasian,M,Accident & Emergency,admit,Med-Surg ICU,Neurological,Neurologic,...,0.694444,0.891892,0.694444,1.282353,-0.375,0.583333,0.666667,0.0,0.0,0.0
4,-0.99984,0.745614,0.968254,Caucasian,F,Accident & Emergency,admit,CSICU,Cardiovascular,Cardiovascular,...,0.111111,-0.297297,0.111111,-0.458824,-0.250,0.833333,0.833333,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.99984,-0.807018,0.473016,Caucasian,M,Accident & Emergency,admit,Med-Surg ICU,Musculoskeletal/Skin,Undefined Diagnoses,...,-0.472222,-0.864865,-0.472222,-0.600000,-0.250,-0.250000,-0.166667,0.0,0.0,0.0
49996,0.99988,-0.412281,0.568254,Caucasian,F,Floor,admit,Med-Surg ICU,Neurological,Neurologic,...,0.833333,0.405405,0.833333,-0.047059,-0.375,-0.250000,-0.333333,0.0,0.0,0.0
49997,0.99992,-0.956140,1.180952,Caucasian,F,Accident & Emergency,transfer,Med-Surg ICU,Sepsis,Cardiovascular,...,-0.833333,-0.405405,-0.833333,-0.376471,NaN,0.333333,0.500000,1.0,0.0,0.0
49998,0.99996,0.245614,0.571429,Caucasian,M,Operating Room / Recovery,admit,Cardiac ICU,Cardiovascular,Cardiovascular,...,0.611111,0.189189,0.611111,-0.352941,-0.125,-0.416667,-0.333333,0.0,0.0,0.0


In [210]:
df_imputed = knn_impute_numerical_columns(df, n_neighbors=100)

In [211]:
df_dropped = df_imputed.copy().dropna(axis=0)
df_dropped.shape
# missing_value_counts(df_dropped)

(50000, 58)

In [212]:
df_onehot = pd.get_dummies(df_dropped)
df_onehot
df_onehot.shape

(50000, 96)

In [213]:
X = df_onehot.loc[:, df_onehot.columns != "hospital_death"]
y = df_onehot[["hospital_death"]]

X.shape

(50000, 95)

In [214]:
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier(n_estimators=1000, max_depth=7, random_state=0, n_jobs=3)
estimator.fit(X, y)

# Get feature importances
feature_importances = estimator.feature_importances_


feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

feature_num = 50
selected_features = feature_importance_df['Feature'][:feature_num]
selected_features

/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


46                 apache_4a_hospital_death_prob
47                      apache_4a_icu_death_prob
10                              gcs_motor_apache
24                                   d1_spo2_min
25                                  d1_sysbp_min
9                                gcs_eyes_apache
26                      d1_sysbp_noninvasive_min
7                            apache_3j_diagnosis
17                             ventilated_apache
16                                   temp_apache
12                             gcs_verbal_apache
27                                   d1_temp_min
22                        d1_mbp_noninvasive_min
21                                    d1_mbp_min
18                                 d1_diasbp_min
6                             apache_2_diagnosis
19                     d1_diasbp_noninvasive_min
20                              d1_heartrate_max
13                             heart_rate_apache
23                               d1_resprate_max
14                  

In [215]:
X = X[selected_features]

In [216]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


In [113]:
def roc_auc_cv(model, X, y):
    cv = RepeatedKFold(n_splits=10, n_repeats=2, random_state=1)
    predicted_probabilities = cross_val_score(model, X, y, scoring="roc_auc", cv=cv, n_jobs=-1, verbose=2)

    return mean(predicted_probabilities)
def calculate_roc_auc(model, X_test, y_test):
    md_probs = model.predict_proba(X_test)
    md_probs = md_probs[:, 1]
    md_auc = roc_auc_score(y_test, md_probs)
    return md_auc

# roc_auc = calculate_roc_auc(model, X_test, y_test)
# print(roc_auc)


In [ ]:

base_classifier = DecisionTreeClassifier(max_depth=7, min_samples_leaf=50, min_samples_split=50)
model = AdaBoostClassifier(base_classifier, n_estimators=100, random_state=42)

# model.fit(X_train, y_train)
roc_auc_cv(model, X, y)

In [ ]:
base_classifier = KNeighborsClassifier(n_neighbors=300)
model = BaggingClassifier(base_classifier, n_estimators=200, random_state=42)

model.fit(X_train, y_train)
calculate_roc_auc(model, X_test, y_test)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators=300, learning_rate=0.3, max_depth=1, random_state=42)
model.fit(X_train, y_train)
calculate_roc_auc(model, X_test, y_test)

In [ ]:
model = GradientBoostingClassifier(n_estimators=200, learning_rate=0.3, max_depth=1, random_state=42)

# Define the parameter grid you want to search
param_grid = {
    'n_estimators': [100, 200, 300],  # You can adjust these values
    'learning_rate': [0.1, 0.2, 0.3],  # You can adjust these values
    'max_depth': [1, 2, 3],  # You can adjust these values
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=3)

# Fit the grid search to your training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and corresponding accuracy score
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy Score:", grid_search.best_score_)

# Get the best model from the grid search
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_


In [222]:
df_catboost = pd.read_csv('train.csv')
df_catboost = pd.get_dummies(df_catboost)
X = df_catboost.loc[:, df_catboost.columns != "hospital_death"]
y = df_catboost[["hospital_death"]]
# X = X[selected_features]


In [223]:
X.shape 

(50000, 95)

In [224]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [231]:
from catboost import CatBoostClassifier, Pool


model = CatBoostClassifier(
    iterations=590,
    depth=5,
    learning_rate=0.09,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=69,
    verbose=20,
    l2_leaf_reg=2,
    border_count=80,
    leaf_estimation_iterations=10,
    # subsample=0.8,
    # min_data_in_leaf=3,
    # colsample_bylevel=0.8,
    
)
# roc_auc_cv(model, X, y)
model.fit(X_train, y_train)
calculate_roc_auc(model, X_test, y_test)

0:	total: 5.86ms	remaining: 3.45s
20:	total: 138ms	remaining: 3.75s
40:	total: 281ms	remaining: 3.77s
60:	total: 423ms	remaining: 3.67s
80:	total: 559ms	remaining: 3.51s
100:	total: 692ms	remaining: 3.35s
120:	total: 816ms	remaining: 3.16s
140:	total: 942ms	remaining: 3s
160:	total: 1.07s	remaining: 2.85s
180:	total: 1.19s	remaining: 2.69s
200:	total: 1.32s	remaining: 2.55s
220:	total: 1.44s	remaining: 2.4s
240:	total: 1.57s	remaining: 2.27s
260:	total: 1.7s	remaining: 2.14s
280:	total: 1.82s	remaining: 2s
300:	total: 1.95s	remaining: 1.87s
320:	total: 2.12s	remaining: 1.77s
340:	total: 2.25s	remaining: 1.64s
360:	total: 2.38s	remaining: 1.51s
380:	total: 2.5s	remaining: 1.37s
400:	total: 2.62s	remaining: 1.23s
420:	total: 2.74s	remaining: 1.1s
440:	total: 2.87s	remaining: 968ms
460:	total: 2.99s	remaining: 837ms
480:	total: 3.11s	remaining: 706ms
500:	total: 3.24s	remaining: 575ms
520:	total: 3.36s	remaining: 445ms
540:	total: 3.48s	remaining: 316ms
560:	total: 3.61s	remaining: 187ms


0.8861038996435868

In [ ]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'iterations': [375],
    'depth': [5],
    'learning_rate': [ 0.09 ],
    'l2_leaf_reg': [1, 2, 3],
    'border_count': [60, 70, 50],  
    'leaf_estimation_iterations': [20, 30, 10],
    
}

# Create the CatBoostClassifier
model = CatBoostClassifier(loss_function='Logloss', eval_metric='AUC', random_seed=42, verbose=20)

# Create a GridSearchCV object
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='roc_auc', n_jobs=3)

# Fit the grid search to your data
grid_search.fit(X, y)

# Print the best hyperparameters
print("Best Parameters: ", grid_search.best_params_)
print("Best AUC Score: ", grid_search.best_score_)


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Define the XGBoost classifier with default parameters
model = XGBClassifier(random_state=42)

# Define the parameter grid for the grid search
param_grid = {
    'n_estimators': [800, 900, 700],
    # 'max_depth': [4, 5],
    'learning_rate': [0.038, 0.039, 0.037],
    # 'min_child_weight': [1, 2],
    'colsample_bytree': [0.8, 0.83, 0.77],
    # 'alpha': [0.1],
    # 'eval_metric': ['logloss'],
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='roc_auc', cv=5, verbose=2)

# Fit the grid search to your data (X, y)
grid_search.fit(X, y)

# Print the best parameters and the corresponding ROC AUC score
print("Best Parameters: ", grid_search.best_params_)
print("Best ROC AUC Score: ", grid_search.best_score_)


In [ ]:
X_train

In [ ]:
X = df.loc[:, df.columns != "hospital_death"]
y = df[["hospital_death"]]

X.shape

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=800,  
    max_depth=4,
    learning_rate=0.038,
    random_state=49,
    min_child_weight=1,
    colsample_bytree=0.78,
    alpha= 0.1,
    eval_metric='logloss',
)
# roc_auc_cv(model, X, y)

# model.fit(X_train, y_train)
roc_auc_cv(model, X, y)
# calculate_roc_auc(model, X_test, y_test)
# print(roc)
# model.get_params()

In [ ]:
X_train.shape

In [ ]:
model = MLPClassifier(
    hidden_layer_sizes=(64,32, 16), 
    activation='relu', 
    max_iter=10, 
    random_state=42, 
    solver='adam',  # You can adjust other hyperparameters as needed
    learning_rate_init=0.001,
    shuffle=True,    
)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=300)
model.fit(X_train, y_train)
roc_auc_cv(model, X, y)


In [ ]:
X.shape

In [ ]:
# model = RandomForestClassifier(n_estimators=100, max_depth=17, min_samples_leaf=100, min_samples_split=200, random_state=0, n_jobs=3)
model.fit(X_train, y_train)
roc_auc_cv(model, X, y)
# calculate_roc_auc(model, X_test, y_test)


In [ ]:
# model = RandomForestClassifier(n_estimators=100, max_depth=17, min_samples_leaf=100, min_samples_split=200, random_state=0, n_jobs=3)
model.fit(X_train, y_train)
# roc_auc_cv(model, X, y)
calculate_roc_auc(model, X_test, y_test)


In [ ]:
model = best_model

In [295]:
df_catboost = pd.read_csv('train.csv')
robust_scale(df_catboost)
df_catboost = knn_impute_numerical_columns(df_catboost, n_neighbors=10)
df_catboost = pd.get_dummies(df_catboost)




In [308]:

X = df_catboost.loc[:, df_catboost.columns != "hospital_death"]
y = df_catboost[["hospital_death"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train.shape


(40000, 95)

In [312]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=590,
    depth=5,
    learning_rate=0.09,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=69,
    verbose=20,
    l2_leaf_reg=2,
    border_count=80,
    leaf_estimation_iterations=20,
    # scale_pos_weight=0.4,
    # subsample=0.8,
    # min_data_in_leaf=1,
    # colsample_bylevel=0.8,
    
)
# roc_auc_cv(model, X, y)
model.fit(X_train, y_train)
calculate_roc_auc(model, X_test, y_test)

0:	total: 7.3ms	remaining: 4.3s
20:	total: 191ms	remaining: 5.18s
40:	total: 436ms	remaining: 5.84s
60:	total: 633ms	remaining: 5.49s
80:	total: 839ms	remaining: 5.27s
100:	total: 1.03s	remaining: 4.98s
120:	total: 1.23s	remaining: 4.77s
140:	total: 1.42s	remaining: 4.52s
160:	total: 1.6s	remaining: 4.28s
180:	total: 1.79s	remaining: 4.04s
200:	total: 1.98s	remaining: 3.84s
220:	total: 2.23s	remaining: 3.73s
240:	total: 2.42s	remaining: 3.51s
260:	total: 2.61s	remaining: 3.29s
280:	total: 2.79s	remaining: 3.07s
300:	total: 2.98s	remaining: 2.86s
320:	total: 3.17s	remaining: 2.65s
340:	total: 3.35s	remaining: 2.45s
360:	total: 3.54s	remaining: 2.24s
380:	total: 3.73s	remaining: 2.05s
400:	total: 3.92s	remaining: 1.85s
420:	total: 4.12s	remaining: 1.66s
440:	total: 4.36s	remaining: 1.47s
460:	total: 4.55s	remaining: 1.27s
480:	total: 4.73s	remaining: 1.07s
500:	total: 4.91s	remaining: 873ms
520:	total: 5.1s	remaining: 675ms
540:	total: 5.28s	remaining: 478ms
560:	total: 5.47s	remaining: 

0.8856506985810823

In [314]:
model.fit(X, y)


0:	total: 8.73ms	remaining: 5.14s
20:	total: 221ms	remaining: 6s
40:	total: 456ms	remaining: 6.11s
60:	total: 679ms	remaining: 5.89s
80:	total: 889ms	remaining: 5.58s
100:	total: 1.16s	remaining: 5.6s
120:	total: 1.4s	remaining: 5.41s
140:	total: 1.64s	remaining: 5.22s
160:	total: 1.86s	remaining: 4.96s
180:	total: 2.12s	remaining: 4.78s
200:	total: 2.32s	remaining: 4.5s
220:	total: 2.53s	remaining: 4.22s
240:	total: 2.76s	remaining: 4s
260:	total: 2.97s	remaining: 3.74s
280:	total: 3.17s	remaining: 3.48s
300:	total: 3.39s	remaining: 3.25s
320:	total: 3.6s	remaining: 3.02s
340:	total: 3.8s	remaining: 2.78s
360:	total: 4s	remaining: 2.54s
380:	total: 4.21s	remaining: 2.31s
400:	total: 4.43s	remaining: 2.09s
420:	total: 4.67s	remaining: 1.87s
440:	total: 4.92s	remaining: 1.66s
460:	total: 5.12s	remaining: 1.43s
480:	total: 5.33s	remaining: 1.21s
500:	total: 5.53s	remaining: 983ms
520:	total: 5.76s	remaining: 764ms
540:	total: 5.97s	remaining: 541ms
560:	total: 6.18s	remaining: 319ms
580:

In [315]:
def generate_predictions_for_model(model, test_file, output_file):
    
    df = pd.read_csv(test_file)
    record_ids = df["RecordID"]
    
    # robust_scale(df)
    # df = knn_impute_numerical_columns(df, n_neighbors=10)
    df = pd.get_dummies(df)
    
    # X_test = df[selected_features] 
    
    X_test = df.loc[:, df.columns != "hospital_death"]
    
    probs = model.predict_proba(X_test)
    probs = probs[:, 1]
    
    # Create a DataFrame for the results
    result = pd.DataFrame({'RecordID': record_ids, 'hospital_death': probs})
    
    # Save the results to a CSV file
    result.to_csv(output_file, index=False, header=["RecordID", "hospital_death"])


In [316]:
generate_predictions_for_model(model, "test.csv", "results74.csv")


In [203]:
import lightgbm as lgb

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

params = {
    'objective': 'binary',  
    'metric': 'binary_logloss', 
    'boosting_type': 'gbdt',  
    'num_leaves': 31,  
    'learning_rate': 0.05,
    'feature_fraction': 0.75,  
    'bagging_fraction': 0.90,  
    'bagging_freq': 5,  # 
    'verbose': 1,
    'max_bin': 255,
    'min_data_in_leaf': 40,  # Example value, adjust as needed
    'min_sum_hessian_in_leaf': 1e-4  ,# Example value, adjust as needed


}

num_round = 211# Number of boosting iterations
lightgbm_model = lgb.train(params, train_data, num_round, valid_sets=[test_data])

y_pred_prob = lightgbm_model.predict(X_test)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)
# roc_auc_cv(lightgbm_model, X, y)



[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3439, number of negative: 36561
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002880 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4884
[LightGBM] [Info] Number of data points in the train set: 40000, number of used features: 86
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.085975 -> initscore=-2.363801
[LightGBM] [Info] Start training from score -2.363801
ROC AUC: 0.8866037897044403


In [ ]:
params = {
    'objective': 'binary',  # Binary classification
    'metric': 'binary_logloss',  # Logarithmic loss for binary classification
    'boosting_type': 'gbdt',  # Gradient boosting decision tree
    'num_leaves': 31,  # Maximum number of leaves in each tree
    'learning_rate': 0.05,  # Step size shrinkage
    'feature_fraction': 0.75,  # Fraction of features to be used in each iteration
    'bagging_fraction': 0.90,  # Fraction of data to be randomly sampled for training
    'bagging_freq': 5,  # Frequency for bagging
    'verbose': 0  # Verbosity level,    
}

num_round = 210  # Number of boosting iterations
lightgbm_model = lgb.train(params, train_data, num_round, valid_sets=[test_data])

y_pred_prob = lightgbm_model.predict(X_test)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)






In [ ]:
def gen_resultfile_lbgm(model, test_file, output_file):
    
    df_test = pd.read_csv(test_file)
    record_ids = df_test["RecordID"]
    
    df_scaled = robust_scale(df_test)
    df_test_imputed = knn_impute_numerical_columns(df_scaled, n_neighbors=100)
    df_test_onehot = pd.get_dummies(df_test_imputed)
    
    X_test = df_test_onehot[selected_features] 
    
    # X_test = df_test_onehot.loc[:, df_test_onehot.columns != "hospital_death"]
    
    probs = model.predict(X_test)
    
    # Create a DataFrame for the results
    result = pd.DataFrame({'RecordID': record_ids, 'hospital_death': probs})
    
    # Save the results to a CSV file
    result.to_csv(output_file, index=False, header=["RecordID", "hospital_death"])

In [ ]:
gen_resultfile_lbgm(lightgbm_model, "test.csv", "results54.csv")

In [ ]:
params = {
    'objective': 'binary',  # Binary classification
    'metric': 'binary_logloss',  # Logarithmic loss for binary classification
    'boosting_type': 'gbdt',  # Gradient boosting decision tree
    'num_leaves': 31,  # Maximum number of leaves in each tree
    'learning_rate': 0.05,  # Step size shrinkage
    'feature_fraction': 0.75,  # Fraction of features to be used in each iteration
    'bagging_fraction': 0.90,  # Fraction of data to be randomly sampled for training
    'bagging_freq': 5,  # Frequency for bagging
    'verbose': 0  # Verbosity level,    
}

num_round = 210  # Number of boosting iterations
lightgbm_model = lgb.train(params, train_data, num_round, valid_sets=[test_data])

y_pred_prob = lightgbm_model.predict(X_test)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)



# Make predictions using the trained model
record_ids = X["RecordID"]
probs = model.predict(X_test)

result = pd.DataFrame({'RecordID': record_ids, 'hospital_death': probs})
    
# Save the results to a CSV file
result.to_csv("results54", index=False, header=["RecordID", "hospital_death"])

In [ ]:
y_pred_prob = lightgbm_model.predict(X_test)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)


In [ ]:
X.shape

In [ ]:
train_data = pd.read_csv('train.csv')  # Replace with your training dataset path
test_data = pd.read_csv('test.csv')    # Replace with your other dataset path


# Create a LightGBM dataset from the training data
train_dataset = lgb.Dataset(X, label=y)

# Define LightGBM parameters
params = {
    'objective': 'binary',  # Binary classification
    'metric': 'binary_logloss',  # Logarithmic loss for binary classification
    'boosting_type': 'gbdt',  # Gradient boosting decision tree
    'num_leaves': 31,  # Maximum number of leaves in each tree
    'learning_rate': 0.05,  # Step size shrinkage
    'feature_fraction': 0.75,  # Fraction of features to be used in each iteration
    'bagging_fraction': 0.90,  # Fraction of data to be randomly sampled for training
    'bagging_freq': 5,  # Frequency for bagging
    'verbose': 0  # Verbosity level,    
}

num_boost_round = 210  # Number of boosting rounds

# Train the LightGBM model on the training dataset
model = lgb.train(params, train_dataset, num_boost_round=num_boost_round)

# Prepare the other dataset for predictions
X_test = test_data  # Preprocess your other dataset as needed

# Make predictions using the trained model
record_ids = X["RecordID"]
probs = lightgbm_model.predict(X_test)

result = pd.DataFrame({'RecordID': record_ids, 'hospital_death': probs})
    
# Save the results to a CSV file
result.to_csv("results54", index=False, header=["RecordID", "hospital_death"])



In [ ]:
test_df = pd.read_csv("test.csv")
robust_scale_df = robust_scale(test_df)
df_test_imputed = knn_impute_numerical_columns(robust_scale_df, n_neighbors=100)
df_test_onehot = pd.get_dummies(df_test_imputed)
X_test = df_test_onehot[selected_features]
X_test.shape

In [ ]:
y_pred_prob = lightgbm_model.predict(X_test)
y_pred_prob


In [ ]:
record_ids = test_df["RecordID"]
result = pd.DataFrame({'RecordID': record_ids, 'hospital_death': probs})
    
# Save the results to a CSV file
result.to_csv("results54.csv", index=False, header=["RecordID", "hospital_death"])